## WaMDaM_Use_Case 3.1: What flow values to use at a site (e.g., below Steward Dam)? 

#### By Adel M. Abdallah, Utah State University, August 2018

This notebook demonstrates basic WaMDaM use cases analysis using scientific Python libraries such as [pandas](https://pandas.pydata.org/) and [plotly](https://plot.ly/).  It reads WaMDaM SQLite data from a published HydroShare Generic Resource, runs SQL script, and them uses Python plotly to visualize the results

This use case identifies five time series and seasonal flow data for the site below Stewart Dam, Idaho



Execute the following cells by pressing `Shift-Enter`, or by pressing the play button 
<img style='display:inline;padding-bottom:15px' src='play-button.png'>
on the toolbar above.


### Steps to reproduce this use case results and plots 

1.[Import python libraries](#Import)   
   
   
2.[Connect to the WaMDaM populated SQLite file](#Connect)    
 
 
3.[Query WaMDaM database for flow time series](#QueryFlowTimeSeries)   
  
  
4.[Plot the compiled time series for Stewart Dam (Figure 11-A)](#PlotFlow12A)  
 
 
5.[Plot the last 15 years to show discrepancy in time series for Stewart Dam (Figure 12-B))](#PlotFlow12B)  
 
 
6.[Pick a a flow source and update the WaMDaM db to reflect "Verified"](#PickaSource)  
 
 
7.[Close the SQLite and WEAP API connections](#Close)  



<a name="Import"></a>
# 1. Import python libraries 

In [17]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib

print 'The needed Python libraries have been imported'

The needed Python libraries have been imported


<a name="Connect"></a>
# 2. Connect to the WaMDaM populated SQLite file 


In [18]:
# 2. Connect to the WaMDaM populated SQLite file 


# Then we can run queries against it within this notebook :)  

# the SQLite file is published here 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/3SQLite_database/BearRiverDatasets_June_2018.sqlite

WaMDaM_SQLite_Name='BearRiverDatasets_June_2018_Final.sqlite'


conn = sqlite3.connect('WaMDaM_SQLite_Name')

print 'Connected to the WaMDaM SQLite file called'+': '+ WaMDaM_SQLite_Name

Connected to the WaMDaM SQLite file called: BearRiverDatasets_June_2018_Final.sqlite


<a name="QueryFlowTimeSeries"></a>
# 3. Query WaMDaM database for flow time series 


In [16]:
# Use Case 3.1Identify_aggregate_TimeSeriesValues.csv
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources



# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase3_1_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/4_Queries_SQL/UseCase3/UseCase3.1/2_Identify_aggregate_TimeSeriesValues.sql

"""

# Read the query text inside the URL
Query_UseCase3_1_text = urllib.urlopen(Query_UseCase3_1_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_1= pd.read_sql_query(Query_UseCase3_1_text, conn)

# uncomment the below line to see the list of attributes
# display (result_df_required)


# Save the datafrom as a csv file into the Jupyter notebook working space
result_df_UseCase3_1.to_csv('UseCases_Results_csv\UseCase3_1.csv', index = False)

print "Queries are done"

DatabaseError: Execution failed on sql '/*
2.2Identify_aggregate_TimeSeriesValues.sql

Use case 2: identify and compare time series and seasonal discharge data across data sources. 
What is the discharge at the node â€œbelow Stewart Damâ€� in Idaho?


Aggregate time series data from daily cfs into cumulative monthly values in acre-feet
If the time series is a Water Year, then convert it to a Calendar year

Here the use of controlled unit names become valuable. 
Users need to check on the unit name to perform a conversion like from cfs to af/month

The query below has two parts: 
The first gets the daily data and aggregate it to monthly and convert its unit af 
The second gets the monthly data and keep it monthly but convert it from cfs to af 
The two parts are needed because of the â€œHavingâ€� function in the second part (to check on the days of the month)

Adel Abdallah
Updated April 3, 2018
*/
--                                               Two select statements will be join togeth

<a name="PlotFlow12A"></a>
# 4. Plot the compiled time series for Stewart Dam (Figure xx)




In [ ]:
# 4. Plot the compiled time series for Stewart Dam (Figure 11-A)


df_TimeSeries=result_df_UseCase3_1
# identify the data for four time series only based on the DatasetAcronym column header 
column_name = "ResourceTypeAcronym"
subsets = df_TimeSeries.groupby(column_name)
data = []

# for each subset (curve), set up its legend and line info manually so they can be edited
subsets_settings = {
    'UDWRFlowData': {
        'dash': 'solid',
        'legend_index': 0,
        'legend_name': 'Utah Division of Water Res.',
        'width':3,
        'color':'rgb(153, 15, 15)'
        },
    'CUAHSI': {
        'dash': 'dash',
        'legend_index': 1,
        'legend_name': 'USGS',
        'width':4,
        'color':'rgb(15, 107, 153)'
        },
    'IdahoWRA': {
        'dash': 'solid',
        'legend_index': 2,
        'legend_name': 'Idaho Department of Water Res.',
        'width':3,
        'color':'rgb(38, 15, 153)'
        },    
    'BearRiverCommission': { # this oone is the name of subset as it appears in the csv file
        'dash': 'dot',     # this is properity of the line (curve)
        'legend_index': 3,   # to order the legend
        'legend_name': 'Bear River Commission',  # this is the manual curve name 
         'width':4,
        'color':'rgb(107, 153, 15)'
        }
    }
    
# This dict is used to map legend_name to original subset name
subsets_names = {y['legend_name']: x for x,y in subsets_settings.iteritems()}

# prepare the scater plot for each curve
for subset in subsets.groups.keys():
    #print subset
    dt = subsets.get_group(name=subset)
    s = go.Scatter(
                    x=dt.CalenderYear.map(lambda z: str(z)[:-3]),
                    y=dt['CumulativeMonthly'],
                    name = subsets_settings[subset]['legend_name'],
                    line = dict(
                        color =subsets_settings[subset]['color'],
                        width =subsets_settings[subset]['width'], 
                        dash=subsets_settings[subset]['dash']
                               ),
                        opacity = 1                                
                  )
    data.append(s)
    
# Legend is ordered based on data, so we are sorting the data based 
# on desired legend order indicarted by the index value entered above
data.sort(key=lambda x: subsets_settings[subsets_names[x['name']]]['legend_index'])

# set up the figure layout parameters
layout = dict(
     #title = "UseCase3.2",
     yaxis = dict(
         title = "Cumulative monthly flow <br> (acre-feet/month)",
         tickformat= ',',
         zeroline=True,
         showline=True,
         ticks='outside',
         ticklen=15,
         #zerolinewidth=4,
         zerolinecolor='#00000f',

         dtick=30000,
                 ),
    xaxis = dict(
         #title = "Time <br> (month/year)",
         #autotick=False,
        tick0='1900-01-01',
        dtick='M180',
        ticks='inside',
        tickwidth=0.5,
        #zerolinewidth=4,
        ticklen=27,
        zerolinecolor='#00000f',
        tickcolor='#000',
        tickformat= "%Y",
       range = ['1920', '2020']

                ),
    legend=dict(
        x=0.2,y=0.9,
        bordercolor='#00000f',
            borderwidth=2


                ),
    autosize=False,
    width=1200,
    height=800,
    margin=go.Margin(l=300, b=150),
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    
    
    font=dict( size=35)
             )
# create the figure object            
fig = dict(data=data, layout=layout)

# plot the figure 
offline.iplot(fig,filename = 'UseCase3.1a_TimeSeries')#,image='png' )       


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  

#plotly.offline.plot(fig, filename = "2.2Identify_aggregate_TimeSeriesValues.html") 

###########################################################################################################
# Have you encounterd the messages below? if not, dont worry about it
# ----------------------------------------------
# Javascript error adding output!
# ReferenceError: Plotly is not defined
# See your browser Javascript console for more details.
# ----------------------------------------------

# Do the follwoing:

# Kernel -> Restart -> Clear all outputs and restart
# Save
# Close browser
# Open browser and run again

print "the plot is generated"

<a name="PlotFlow12B"></a>
# 5. Plot the last 15 years to show discrepency in time series for Stewart Dam (Figure xx-B)




In [ ]:
# 5. Plot the last 15 years to show discrepency in time series for Stewart Dam (Figure 12-b)

# Use Case 2.2bIdentify_aggregate_TimeSeriesValues.py
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources

# Adel Abdallah
# November 16, 2017

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from random import randint
import pandas as pd

## read the input data from GitHub csv file which is a direct query output for this  query:
# 3.2Identify_aggregate_TimeSeriesValues.sql


# identify the data for four time series only based on the DatasetAcronym column header 
column_name = "ResourceTypeAcronym"
subsets = df_TimeSeries.groupby(column_name)
data = []

# for each subset (curve), set up its legend and line info manually so they can be edited

subsets_settings = {
    'UDWRFlowData': {
        'symbol': "star",
        'legend_index': 0,
        'legend_name': 'Utah Division of Water Res.',
        'width':2,
        'size' :7,
        'color':'rgb(153, 15, 15)',
        'mode': 'lines+markers'
        },
    'CUAHSI': {
        'symbol': "square",
        'legend_index': 1,
         'size' :10,
        'legend_name': 'CUAHSI',
        'width':3,
        'color':'rgb(15, 107, 153)',
        'show_legend': False,
        },
    'IdahoWRA': {
        'symbol': "triangle-down",
        'legend_index': 2,
         'size' :6,
        'legend_name': 'Idaho Department of Water Res.',
        'width':3,
        'color':'rgb(38, 15, 153)'
        },    
    'BearRiverCommission': { # this one is the name of subset as it appears in the csv file
        'symbol': 106,     # this is property of the line (curve)
                'size' :6,

        'legend_index': 3,   # to order the legend
        'legend_name': "Bear River Commission",  # this is the manual curve name 
         'width':4,
        'color':'rgb(107, 153, 15)'
        }
    }
    
# This dict is used to map legend_name to original subset name
subsets_names = {y['legend_name']: x for x,y in subsets_settings.iteritems()}

# prepare the scater plot for each curve
for subset in subsets.groups.keys():
    print subset
    dt = subsets.get_group(name=subset)
    s = go.Scatter(
        x=dt.CalenderYear.map(lambda z: str(z)[:-3]),
        y=dt['CumulativeMonthly'],
        name = subsets_settings[subset]['legend_name'],       
        opacity = 1,
        
        # Get mode from settings dictionary, if there is no mode
        # defined in dictinoary, then default is markers.
        mode = subsets_settings[subset].get('mode', 'markers'),
        
        # Get legend mode from settings dictionary, if there is no mode
        # defined in dictinoary, then default is to show item in legend.
        showlegend = subsets_settings[subset].get('show_legend', True),
        
        marker = dict(
            size =subsets_settings[subset]['size'],
            color = '#FFFFFF',      # white
            symbol =subsets_settings[subset]['symbol'],
            line = dict(
                color =subsets_settings[subset]['color'],
                width =subsets_settings[subset]['width'], 
                ),
            ),
            
        line = dict(
            color =subsets_settings[subset]['color'],
            width =subsets_settings[subset]['width'], 
            ),
        )
    
    data.append(s)
    
# Legend is ordered based on data, so we are sorting the data based 
# on desired legend order indicated by the index value entered above
data.sort(key=lambda x: subsets_settings[subsets_names[x['name']]]['legend_index'])

# set up the figure layout parameters
layout = dict(
     #title = "UseCase3.2",
     yaxis = dict(
         title = "Cumulative monthly flow <br> (acre-feet/month)",
         tickformat= ',',
         zeroline=True,
         showline=True,
         ticks='outside',
         ticklen=15,
         #zerolinewidth=4,
         zerolinecolor='#00000f',
         range = ['0', '6000'],
         dtick=1000,
                 ),
    xaxis = dict(
         #title = "Time <br> (month/year)",
         #autotick=False,
        tick0='1994-01-01',
        showline=True,
        dtick='M12',
        ticks='outside',
        tickwidth=0.5,
        #zerolinewidth=4,
        ticklen=27,
        #zerolinecolor='#00000',
        tickcolor='#000',
        tickformat= "%Y",
        range = ['1994', '2000']
                ),
    legend=dict(
        x=0.3,y=1,
        bordercolor='#00000f',
            borderwidth=2


                ),
    autosize=False,
    width=1200,
    height=800,
    margin=go.Margin(l=300, b=150),
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    
    
    font=dict( size=35)
             )
             
# create the figure object            
fig = dict(data=data, layout=layout)

# plot the figure 
#py.iplot(fig, filename = "2.2bIdentify_aggregate_TimeSeriesValues")       


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'UseCase3.1b_TimeSeries')#,image='png' )       
print "the plot is generated"

<a name="PickaSource"></a>
# 6. Pick a a flow source and update the db to reflect "Verified"

This "Update" SQL query allows users to update the Mappings table to indicate a "verified" DataValue. 
A verified record set to True indicates that the user has verified, curated, checked, or selected this 
data value as ready to be used for models. A verified record can then be used from an automated script to 
serve data to models. Its particularly useful when the same set of controlled object type, attribute, and instances names 
return multiple data values from different sources with potentially similar or different values due to many factors.


In [ ]:
# 6. Pick a a flow source and update the db to reflect "Verified"

# scenario_name_data = subsets.get_group(name='Base case')
# print scenario_name_data
# Get a cursor object

SQL_update = """
UPDATE Mappings 

SET Verified= 'True'
WHERE  MappingID in

(SELECT Mappings.MappingID FROM Mappings

-- Join the Mappings to get their Attributes
LEFT JOIN "Attributes"
ON Attributes.AttributeID= Mappings.AttributeID

-- Join the Attributes to get their ObjectTypes
LEFT JOIN  "ObjectTypes"
ON "ObjectTypes"."ObjectTypeID"="Attributes"."ObjectTypeID"

-- Join the Mappings to get their Instances   
LEFT JOIN "Instances" 
ON "Instances"."InstanceID"="Mappings"."InstanceID"

-- Join the Mappings to get their ScenarioMappings   
LEFT JOIN "ScenarioMappings"
ON "ScenarioMappings"."MappingID"="Mappings"."MappingID"

-- Join the ScenarioMappings to get their Scenarios   
LEFT JOIN "Scenarios"
ON "Scenarios"."ScenarioID"="ScenarioMappings"."ScenarioID"

-- Join the Scenarios to get their MasterNetworks   
LEFT JOIN "MasterNetworks" 
ON "MasterNetworks"."MasterNetworkID"="Scenarios"."MasterNetworkID"

where 
ObjectTypes.ObjectType='Site'  

AND "Instances"."InstanceName"="10046500.MONBEAR RIVER BL STEWART DAM NR MONTPELIER IDAHO"  

AND AttributeName='Delivered volume per month'

AND ScenarioName='Existing data'

AND MasterNetworkName='UDWRFlowData')
"""

cur = conn.cursor()

res = cur.execute(SQL_update)

print 'updated'

<a name="Close"></a>
# 7. Close the SQLite connection


In [ ]:
conn.close()

print 'Connection to SQLite engine is disconnected'

# The End :) Congratulations